In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
file1 = open(r'/Users/siddhantbhanot/Developer/Sentiment-Analysis/Hinglish-tweet-classification/6) Models/stopwords-hi.txt', 'r',encoding="utf8")
Lines = file1.readlines()

In [5]:
hindi_stop = []
# Strips the newline character
for line in Lines:
    hindi_stop.append(line.strip())

In [8]:
df1 = pd.read_excel(r'/Users/siddhantbhanot/Developer/Sentiment-Analysis/Hinglish-tweet-classification/3) Dataset/3) TRAC 2 back_trans/FinalHinDevTrac2_BackTran.xlsx', index_col = 0)
df2 = pd.read_excel(r'/Users/siddhantbhanot/Developer/Sentiment-Analysis/Hinglish-tweet-classification/3) Dataset/3) TRAC 2 back_trans/FinalHinTrainTrac2_BackTran.xlsx', index_col = 0)

In [9]:
df1.head()

,ID,data,Sub-task A,Sub-task B
0,C38.9,बक्ल साक्षात्कारकर्ताओं कुछ जाड़ा हाय खुले दिम...,OAG,NGEN
1,C4.1510,भैया शादी मैं ज़रूर आना फिल्म का कथानक था क्या...,NAG,NGEN
2,C19.95,धारा 375 हैं क्या ??? .... ये बताते कम से कम को,NAG,NGEN
3,C4.281,कबीर सिंह मारा हुई इससे पता चलता है आजकल के लो...,OAG,NGEN
4,C4.216,मेन इतनी कम नापसंद क्भी न्ही देखी,NAG,NGEN


In [10]:
df2.head()

,ID,data,Sub-task A,Sub-task B
0,C4.131,बॉलीवुड फिल्म देखने के समय तर्क घर में चोरके ए...,NAG,NGEN
1,C4.638,चुटिया फिल्म ...,NAG,NGEN
2,C38.598,हमारे जाट बन्दे का खेने का अर्थ था मार्च डालुग...,OAG,NGEN
3,C4.2101.1,@फेमिनिस्म कैंसर की * नारीवादी ये साही हैं संय...,OAG,NGEN
4,C29.14.2,अमृत ​​आनंद अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


In [11]:
df1['Sub-task A'].value_counts()

Sub-task A
CAG    602
OAG    582
NAG    578
Name: count, dtype: int64

In [12]:
df2['Sub-task A'].value_counts()

Sub-task A
CAG    2453
NAG    2245
OAG    1820
Name: count, dtype: int64

In [14]:
df = pd.concat([df1,df2],ignore_index=True)

In [15]:
df.head()

,ID,data,Sub-task A,Sub-task B
0,C38.9,बक्ल साक्षात्कारकर्ताओं कुछ जाड़ा हाय खुले दिम...,OAG,NGEN
1,C4.1510,भैया शादी मैं ज़रूर आना फिल्म का कथानक था क्या...,NAG,NGEN
2,C19.95,धारा 375 हैं क्या ??? .... ये बताते कम से कम को,NAG,NGEN
3,C4.281,कबीर सिंह मारा हुई इससे पता चलता है आजकल के लो...,OAG,NGEN
4,C4.216,मेन इतनी कम नापसंद क्भी न्ही देखी,NAG,NGEN


In [16]:
df['Sub-task A'].value_counts()

Sub-task A
CAG    3055
NAG    2823
OAG    2402
Name: count, dtype: int64

Removing all emoticons from the text

In [17]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [18]:
df['data'] = df['data'].apply(lambda x :remove_emojis(x))

In [20]:
from sklearn.model_selection import train_test_split

X = df['data']
y = df['Sub-task A']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [21]:
print(X_train.shape[0])
print(X_test.shape[0])

5796
2484


### Applying TF-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3,4), stop_words = hindi_stop)

X_train_final = vectorizer.fit_transform(X_train)
X_test_final = vectorizer.transform(X_test)

print(X_train.shape)
print(X_train_final.shape)

/Users/siddhantbhanot/Developer/Sentiment-Analysis/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:543: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


(5796,)
(5796, 79731)


In [32]:
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

### Applying SVM

In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf']} 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

grid.fit(X_train_final, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.384 total time=  20.7s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.385 total time=  20.9s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.383 total time=  20.8s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.390 total time=  20.5s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.385 total time=  20.5s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.374 total time=  20.2s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.374 total time=  20.7s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.374 total time=  20.6s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.374 total time=  19.9s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.374 total time=  19.9s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.374 total time=  19.7s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [30]:
import joblib

# Save the model to a file
joblib.dump(grid.best_estimator_, 'svm_model.pkl')

['svm_model.pkl']

In [31]:
grid_predictions = grid.predict(X_test_final)

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, grid_predictions))
print(confusion_matrix(y_test, grid_predictions))
print(classification_report(y_test, grid_predictions))

0.7431561996779388
[[702 128  59]
 [118 677  48]
 [170 115 467]]
              precision    recall  f1-score   support

         CAG       0.71      0.79      0.75       889
         NAG       0.74      0.80      0.77       843
         OAG       0.81      0.62      0.70       752

    accuracy                           0.74      2484
   macro avg       0.75      0.74      0.74      2484
weighted avg       0.75      0.74      0.74      2484



### Applying XG Boost

In [29]:
from xgboost import XGBClassifier

clf1 = XGBClassifier()
clf1.fit(X_train_final, y_train)

predictions = clf1.predict(X_test_final)

[11:18:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.6896135265700483
[[664 143  82]
 [140 645  58]
 [218 130 404]]
              precision    recall  f1-score   support

         CAG       0.65      0.75      0.69       889
         NAG       0.70      0.77      0.73       843
         OAG       0.74      0.54      0.62       752

    accuracy                           0.69      2484
   macro avg       0.70      0.68      0.68      2484
weighted avg       0.70      0.69      0.69      2484



### Applying Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

clf2 = LogisticRegression(solver='lbfgs', max_iter=10000)
clf2.fit(X_train_final, y_train)

predictions = clf2.predict(X_test_final)

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.711352657004831
[[676 139  74]
 [126 667  50]
 [203 125 424]]
              precision    recall  f1-score   support

         CAG       0.67      0.76      0.71       889
         NAG       0.72      0.79      0.75       843
         OAG       0.77      0.56      0.65       752

    accuracy                           0.71      2484
   macro avg       0.72      0.71      0.71      2484
weighted avg       0.72      0.71      0.71      2484



#### Using pipeline to implement TFIDF and apply linear SVM

TFIDF Vectorizer

In [ ]:
# !pip install mlxtend

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.preprocessing import DenseTransformer

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ("densifier", DenseTransformer()),
                      ('clf', svm.SVC()),
                    ])
text_clf.fit(X_train, y_train)

In [ ]:
predictions = text_clf.predict(X_test)

Results

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

#### Using pipeline to implement TFIDF and XG boost

In [ ]:
# !pip install xgboost

In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.preprocessing import DenseTransformer

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ("densifier", DenseTransformer()),
                      ('clf', XGBClassifier(random_state = 13))
                    ])
text_clf.fit(X_train, y_train)

In [ ]:
predictions = text_clf.predict(X_test)

Result

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

#### Using pipeline to implement TFIDF and apply linear Logistic Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.preprocessing import DenseTransformer

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ("densifier", DenseTransformer()),
                      ('clf', LogisticRegression())
                    ])
text_clf.fit(X_train, y_train)

In [ ]:
predictions = text_clf.predict(X_test)

Result

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

#### Using pipeline to implement TFIDF (trigram) and apply linear SVM

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3,4))

In [17]:
X_train.shape

(5796,)

In [18]:
X_train_final = vectorizer.fit_transform(X_train)

In [19]:
print(X_train_final.shape)

(5796, 79731)


In [20]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X_train_final, y_train)

LinearSVC()

In [21]:
X_test_final = vectorizer.transform(X_test)

In [22]:
predictions = clf.predict(X_test_final)

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.7355072463768116
[[667 136  86]
 [120 664  59]
 [159  97 496]]
              precision    recall  f1-score   support

         CAG       0.71      0.75      0.73       889
         NAG       0.74      0.79      0.76       843
         OAG       0.77      0.66      0.71       752

    accuracy                           0.74      2484
   macro avg       0.74      0.73      0.73      2484
weighted avg       0.74      0.74      0.73      2484



In [27]:
from xgboost import XGBClassifier

In [28]:
clf = XGBClassifier()

In [29]:
clf.fit(X_train_final, y_train)

C:\Users\siddh\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:30:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
X_test_final = vectorizer.transform(X_test)

In [31]:
predictions = clf.predict(X_test_final)

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.6896135265700483
[[664 143  82]
 [140 645  58]
 [218 130 404]]
              precision    recall  f1-score   support

         CAG       0.65      0.75      0.69       889
         NAG       0.70      0.77      0.73       843
         OAG       0.74      0.54      0.62       752

    accuracy                           0.69      2484
   macro avg       0.70      0.68      0.68      2484
weighted avg       0.70      0.69      0.69      2484



In [37]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', max_iter=10000)

In [38]:
clf.fit(X_train_final, y_train)

LogisticRegression(max_iter=10000)

In [39]:
predictions = clf.predict(X_test_final)

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.711352657004831
[[676 139  74]
 [126 667  50]
 [203 125 424]]
              precision    recall  f1-score   support

         CAG       0.67      0.76      0.71       889
         NAG       0.72      0.79      0.75       843
         OAG       0.77      0.56      0.65       752

    accuracy                           0.71      2484
   macro avg       0.72      0.71      0.71      2484
weighted avg       0.72      0.71      0.71      2484

